In [1]:
import pandas as pd
import re # Регулярные выражения.
import requests # Загрузка новостей с сайта.
from bs4 import BeautifulSoup # Превращалка html в текст.
import pymorphy2 # Морфологический анализатор.
import datetime # Новости будем перебирать по дате.
from collections import Counter # Не считать же частоты самим.
import math # Корень квадратный.
import numpy

In [2]:
train = pd.read_csv('data/train.csv', sep='\t', index_col='id')

In [3]:
test = pd.read_csv('data/test.csv', sep='\t', index_col='id')

In [4]:
other = pd.read_csv('data/other.csv', sep='\t')

In [5]:
train.head()

,name,description,target
id,,,
0,Заведующий отделом/секцией в магазин YORK (Уру...,<p><strong>В НОВЫЙ МАГАЗИН YORK (хозтовары) пр...,1
1,Наладчик станков и манипуляторов с ПУ,Обязанности:работа на токарных станках с ЧПУ T...,0
2,Разработчик С++ (Криптограф),<strong>Требования:</strong> <ul> <li>Опыт про...,0
3,Фрезеровщик,<p>Условия:</p> <ul> <li>На работу вахтовым ме...,0
4,Мерчендайзер/продавец-консультант,<p><strong>Компания Палладиум Стандарт - призн...,1


In [6]:
test.head()

,name,description
id,,
200000,Дизайнер-консультант мебели,<p><strong>Обязанности:</strong></p> <ul> <li>...
200001,Продавец-консультант (ТЦ на Пушкина),<p><strong>Обязанности</strong>:</p> <p>∙ конс...
200002,Менеджер по продажам,<p>Торговый Дом «Форт» это ведущая компания Пе...
200003,Продавец-консультант в магазин одежды (ТЦ Волн...,<p><strong>Требуются продавцы консультанты в м...
200004,Специалист по охране труда,<strong>Обязанности:</strong> <ul> <li> <p>осу...


In [7]:
other.head()

,name,description
0,Специалист научно-производственного отдела,"<p>Образование - Среднее специальное, высшее</..."
1,Оператор по отгрузке товара в 1С (ТЗ),<p><strong>Обязанности:</strong></p> <ul> <li>...
2,Менеджер по персоналу,<strong>Обязанности:</strong> <ul> <li>Подбор ...
3,Ведущий бухгалтер по учёту заработной платы,<strong>Обязанности:</strong> <ul> <li> <p>Нач...
4,Инженер-расчетчик в строительный отдел,<p><strong>Требования:</strong></p> <ul> <li>у...


In [8]:
y = train.target

In [9]:
def preproc_data(df_input):
    df_output = df_input.copy()
    
    df_output = df_output.drop('target', axis=1)
#     names = list(set(df_output['name']))
#     df_output['name'] = df_output.name.apply(lambda r: name.index(r))

#     df_output = df_output.join(other, on=['name', 'name'], lsuffix='', rsuffix='_other')
#     df_output['description'] = BeautifulSoup(df_output['description'], "lxml").get_text()
#     df_output['description_other'] = BeautifulSoup(df_output['description_other'], "lxml").get_text()
#     df_output = pd.get_dummies(df_output, columns=[
#          'job',
#          'month',
#          'day_of_week',
#          'marital',
#          'education',
#          'default',
#          'housing',
#          'loan',
#          'contact',
#          'poutcome'
#           'duration_class'
#     ])
    
    return df_output

class JobModel:
    jobs=[]
    dictionaries=[]
    
    def __init__(self, df_jobs):
        self.jobs = df_jobs.apply(lambda x: x['name'] + '\n\n' +\
                             re.sub(r'\<[^>]*\>', '', x['description']), axis=1).values
        self.morph=pymorphy2.MorphAnalyzer()
         
    posConv={'ADJF':'_ADJ','NOUN':'_NOUN','VERB':'_VERB'}
    def getJobDictionary(self, text, needPos=None):
        words=[a[0] for a in re.findall("([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)", text)]
        reswords=[]
    
        for w in words:
            wordform=self.morph.parse(w)[0]
            try:
                if wordform.tag.POS in ['ADJF', 'NOUN', 'VERB']:
                    if needPos!=None:
                        reswords.append(wordform.normal_form+self.posConv[wordform.tag.POS])
                    else:
                        reswords.append(wordform.normal_form)
            except:
                pass
            
        stat=Counter(reswords)
        return stat
    
    def calcJobDictionaries(self, needPos=None):
        self.dictionaries=[]
        counter = 0
        pCount = 0
        for a in self.jobs:
            self.dictionaries.append(self.getJobDictionary(a, needPos))
            counter=counter+1
            if counter > 1000:
                pCount=pCount+counter
                counter = 0
                print(pCount)

In [10]:
def text_to_vec(dct, model, size):
    text_vec = np.zeros((size,), dtype="float32")
    n_words = 0

    index2word_set = set(model.index2word)
    for word in dct.keys():
        if word in index2word_set:
            n_words = n_words + 1
            text_vec = np.add(text_vec, model[word]) 
    
    if n_words != 0:
        text_vec /= n_words
    return text_vec

In [11]:
target_model = JobModel(train[train['target']==1])
target_model.calcJobDictionaries(True)

1001
2002
3003
4004
5005
6006
7007
8008
9009
10010
11011
12012
13013
14014
15015
16016
17017
18018
19019
20020
21021
22022
23023
24024
25025
26026
27027
28028
29029
30030
31031
32032
33033
34034
35035
36036
37037
38038
39039
40040
41041
42042
43043
44044
45045
46046
47047
48048
49049
50050
51051
52052
53053
54054
55055
56056
57057
58058
59059
60060
61061
62062
63063
64064
65065
66066
67067
68068
69069
70070
71071
72072
73073
74074
75075
76076
77077
78078
79079
80080
81081
82082
83083
84084
85085
86086
87087
88088
89089
90090
91091
92092
93093


In [12]:
print(test.info())
test_model = JobModel(test)
test_model.calcJobDictionaries(True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170179 entries, 200000 to 370178
Data columns (total 2 columns):
name           170179 non-null object
description    170179 non-null object
dtypes: object(2)
memory usage: 3.9+ MB
None
1001
2002
3003
4004
5005
6006
7007
8008
9009
10010
11011
12012
13013
14014
15015
16016
17017
18018
19019
20020
21021
22022
23023
24024
25025
26026
27027
28028
29029
30030
31031
32032
33033
34034
35035
36036
37037
38038
39039
40040
41041
42042
43043
44044
45045
46046
47047
48048
49049
50050
51051
52052
53053
54054
55055
56056
57057
58058
59059
60060
61061
62062
63063
64064
65065
66066
67067
68068
69069
70070
71071
72072
73073
74074
75075
76076
77077
78078
79079
80080
81081
82082
83083
84084
85085
86086
87087
88088
89089
90090
91091
92092
93093
94094
95095
96096
97097
98098
99099
100100
101101
102102
103103
104104
105105
106106
107107
108108
109109
110110
111111
112112
113113
114114
115115
116116
117117
118118
119119
120120
121121
122122
123123
124124
1251

In [13]:
from gensim.models.word2vec import Word2Vec # Собственно модель.
from gensim.models.word2vec import LineSentence # Выравнивание текста по предложениям.
from gensim.models import KeyedVectors # Семантические вектора.

model_w2v = KeyedVectors.load_word2vec_format('data/news_upos_cbow_600_2_2018.vec')

In [14]:
from sklearn.cluster import DBSCAN
import umap
import numpy as np

In [16]:
umap_jobs=umap.UMAP()
w2v_train_vectors=np.array([text_to_vec(dct, model_w2v, 600) for dct in target_model.dictionaries])
w2v_test_vectors=np.array([text_to_vec(dct, model_w2v, 600) for dct in test_model.dictionaries])

In [ ]:
w2v_vectors = np.concatenate((w2v_train_vectors, w2v_test_vectors))

umaped_v = umap_jobs.fit_transform(w2v_vectors)
db=DBSCAN(eps=0.2, min_samples=3)
db_clusters=db.fit_predict(umaped_v)
ax=pd.DataFrame(umaped_v).plot.scatter(x=0, y=1, c=db_clusters, cmap=plt.cm.Set1)

In [ ]:
target_model.jobs